<a href="https://colab.research.google.com/github/Yuuraa/Pstage2-KLUE/blob/main/%5B_Boost_Camp_%5D_KLUE_%EC%B1%97%EB%B4%87_%EB%A7%8C%EB%93%A4%EC%96%B4%EB%B3%B4%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 2.1MB 7.9MB/s 
     |████████████████████████████████| 3.3MB 48.9MB/s 
     |████████████████████████████████| 870kB 49.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=edd2f0105e0dff822d588073892143fb2af927cefe4f65fc24e627ef21349035
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [2]:
!git clone https://github.com/songys/Chatbot_data.git

Cloning into 'Chatbot_data'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 42 (delta 14), reused 0 (delta 0), pack-reused 18
Unpacking objects: 100% (42/42), done.


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
from transformers import AutoTokenizer, AutoModel, AutoConfig

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

In [ ]:
# config = AutoConfig.from_pretrained("monologg/koelectra-base-v3-discriminator")
model = AutoModel.from_pretrained("monologg/koelectra-base-v3-discriminator")

In [ ]:
train_df = pd.read_csv('/content/Chatbot_data/ChatbotData .csv')

In [ ]:
train_df.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [8]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
from tqdm.notebook import tqdm
from sklearn.metrics.pairwise import cosine_similarity



class SimpleChatbot:
    def __init__(self, pretrained_model, limit=0.6):
        self.tokenizer = AutoTokenizer.from_pretrained(pretrained_model)
        self.model = AutoModel.from_pretrained(pretrained_model)
        self.encoded_questions = []
        self.limit = limit


    def __call__(self, query):
        answer_id = self.get_closest_answer(query)
        if answer_id == -1:
            return "잘 이해하지 못했어요. 다시 말해줄래요?"
        else:
            return self.train_df['A'].iloc()


    def setup_model(self, train_file):
        """
        - train_file: csv 형식으로, 'Q' column에 질문들을, 'A' column에 답변들을 저장해 둔 파일
        - 
        """
        self.train_df = pd.read_csv(train_file)
        tokenized_sentences = []
        print("Tokenizing sentences...")
        for question in tqdm(self.train_df['Q']):
            tokenized_sentences.append(self.tokenizer(
                                            question,
                                            return_tensors="pt",
                                            truncation=True,
                                            add_special_tokens=True,
                                            max_length=128
                                        ))
        print("Total number of sentences", len(tokenized_sentences))
        print("Encoding sentences...")
        with torch.no_grad():
            for tokenized_sentence in tqdm(tokenized_sentences):
                encoded_sentence = self.model(
                    input_ids = tokenized_sentence['input_ids'],
                    attention_mask = tokenized_sentence['attention_mask'],
                    token_type_ids = tokenized_sentence['token_type_ids']
                ).last_hidden_state[:, 0, :].detach().cpu().numpy()
                self.encoded_questions.append(encoded_sentence)
        print("Setup done")


    def __len__(self):
        if not self.encoded_questions:
            return "No encoded questions. Setup chatbot with question and answer data"
        else:
            return len(self.encoded_questions)


    def get_closest_answer(self, query):
        tokenized_query = self.tokenizer(query,
                                         return_tensors="pt",
                                         truncation=True,
                                         add_special_tokens=True,
                                         max_length=128
                                         )
        encoded_query = self.model(
                                input_ids=tokenized_query['input_ids'],
                                attention_mask=tokenized_query['attention_mask'],
                                token_type_ids=tokenized_query['token_type_ids'],
                                ).last_hidden_state[:,0,:].detach().cpu().numpy()
                        
        cos_sim = cosine_similarity(encoded_query, dataset_cls_hidden)
        if max(cos_sim) > self.limit:
            return np.argmax(cos_sim)
        else:
            return -1


    


In [9]:
chatbot = SimpleChatbot("monologg/koelectra-base-v3-discriminator")

In [10]:
chatbot.setup_model('/content/Chatbot_data/ChatbotData .csv')

11823


In [11]:
chatbot("요즘 힘들어")

NameError: ignored

In [ ]:
chatbot("오늘 날씨가 좋네")

In [ ]:
chatbot("오늘 많이 힘들었어")

In [14]:
train_df = pd.read_csv("/content/Chatbot_data/ChatbotData .csv")
train_df["encoded"] = chatbot.encoded_questions

In [18]:
train_df.to_csv("/content/ChatbotData.csv")

In [ ]:
from flask-ngrok import Flask

In [19]:
train_df = pd.read_csv("/content/drive/MyDrive/ChatbotData.csv")

In [20]:
train_df.head()

,Unnamed: 0,Q,A,label,encoded
0,0,12시 땡!,하루가 또 가네요.,0,[[-2.23610520e-01 -4.10217881e-01 -3.01733017e...
1,1,1지망 학교 떨어졌어,위로해 드립니다.,0,[[-5.31819761e-02 -3.29766363e-01 -2.46364415e...
2,2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0,[[ 9.16604400e-02 -3.55314821e-01 -2.85758853e...
3,3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0,[[ 8.37477893e-02 -3.27783465e-01 -2.91657716e...
4,4,PPL 심하네,눈살이 찌푸려지죠.,0,[[-1.67375490e-01 -2.41622522e-01 -4.60954815e...
